In [34]:
# Example: Former Subdivision

import pandas as pd

# Merging the same infobox
excel_path = pd.ExcelFile('D:\Kuliah\Skripsi\Ontologi-Sejarah-Indonesia\MichaelDawBalma\Code\infoboxes\infobox-orla.xlsx')
sheet_name = pd.read_excel(excel_path, 'military conflict1')
sheet_name2 = pd.read_excel(excel_path, 'Military Conflict2')
sheet_name3 = pd.read_excel(excel_path, 'military conflict')
csv_file_path = 'military-conflict-final.csv'

merged_df = pd.concat([sheet_name, sheet_name2, sheet_name3])

merged_df.to_csv(csv_file_path, index=False)

In [35]:
# Clean the merged Former Subdivision data///
import csv
military_conflict_clean = []

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)

    for row in reader:
        military_conflict_clean.append(dict(row))

In [36]:
import geopy

def get_coordinates(place):
    geolocator = geopy.Nominatim(user_agent='indonesian-history-ontology')
    lct = geolocator.geocode(place, geometry='wkt')
    if lct:
        return lct.latitude, lct.longitude, lct.raw['geotext']
    else:
        return -0.789275, 113.921327, ''


for index, value in enumerate(military_conflict_clean):
    latitude, longitude, location = get_coordinates(value['place'])

    # Add latitude and longitude into the dictionary  
    value['latitude'] = latitude
    value['longitude'] = longitude
    value['location'] = location

In [65]:
'''
Merge column date_start-year_start and date_end-year_end into merged_date_start and merged_date_end
https://raw.githubusercontent.com/w3c/sdw/gh-pages/time/rdf/time.ttl (time:inXSDDate)
'''

month_list = {
    'Januari' : '01',
    'Februari' : '02',
    'Maret' : '03',
    'April' : '04',
    'Mei' : '05',
    'Juni' : '06',
    'Juli' : '07',
    'Agustus' : '08',
    'September' : '09',
    'Oktober' : '10',
    'November' : '11',
    'Desember' : '12',
}

defaultDateStart = '01-01'
defaultDateEnd = '12-31'

def check_date_format(date):
    # returns to dd format
    if len(date) == 1:
        return '0' + f'{date}'
    return date

for index, value in enumerate(military_conflict_clean):
    
    day = ''
    month = ''
    year = ''
    dateStartUsed = ''
    dateEndUsed = ''
    dt = ''
    dtSt = ''
    dtEnd = ''

    split_date = value['date'].split("-")
    
    # if it does not have end date
    if len(split_date) == 1 :
        dt = split_date[0].strip().split(" ")
        # if the dt is only the year
        if len(dt) == 1 :
            dateStartUsed = f"{dt[0]}-{defaultDateStart}"
            dateEndUsed = f"{dt[0]}-{defaultDateEnd}"
        else:
            day = dt[0]
            month = month_list[dt[1]]
            year = dt[2]
            
            day = check_date_format(day)
            dateStartUsed = f'{year}-{month}-{day}'
            dateEndUsed = dateStartUsed
            
    # if it does have end date
    else :
        dtSt = split_date[0].strip().split(" ")
        dtEnd = split_date[1].strip().split(" ")
        
        # end date were in good format
        day = dtEnd[0]
        month = month_list[dtEnd[1]]
        year = dtEnd[2]
        # change into dd format
        day = check_date_format(day)
        dateEndUsed = f"{year}-{month}-{day}"
        
        # start date and it is year only
        if len(dtSt) == 1:
            dateStartUsed = f"{dtSt[0]}-{defaultDateStart}"
        # start date without year; get end-date's year
        elif len(dtSt) == 2:
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtEnd[2]}-{month}-{day}"
        else :
            day = check_date_format(dtSt[0])
            month = month_list[dtSt[1]]
            dateStartUsed = f"{dtSt[2]}-{month}-{day}"
    
    value['start-date'] = dateStartUsed
    value['end-date'] = dateEndUsed


In [59]:
import wikipedia

def retrieve_paragraph(article_title):
    wikipedia.set_lang("id")
    return wikipedia.summary(article_title, sentences=5, auto_suggest=False)

def get_url_wikipedia(article_title):
    wikipedia.set_lang("id")
    page = wikipedia.page(article_title, auto_suggest=False)
    return page.url


In [62]:
for index, data in enumerate(military_conflict_clean[:10]):
    summary = retrieve_paragraph(data['page_title'])
    wikiurl = get_url_wikipedia(data['page_title'])
    data['summary'] = summary
    data['wikiurl'] = wikiurl

In [63]:
for index, data in enumerate(military_conflict_clean[10:]):
    summary = retrieve_paragraph(data['page_title'])
    wikiurl = get_url_wikipedia(data['page_title'])
    data['summary'] = summary
    data['wikiurl'] = wikiurl

In [66]:
fieldnames = military_conflict_clean[0].keys()

with open('military-conflict-final-cleaned.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

    writer.writerows(military_conflict_clean)